In [1]:
import pandas as pd
import pyspark 
from pyspark.sql.functions import col
from pyspark.sql import SQLContext

In [3]:
spark = (pyspark.sql.SparkSession.builder
    .master("local")
    .getOrCreate())

In [17]:
video_reviews_df = spark.read.json('data/reviews_Amazon_Instant_Video.json')
meta_video_reviews = spark.read.json('data/meta_Amazon_Instant_Video.json')
comic_reviews_df = spark.read.json('data/comic_reviews_wtitle.json')

In [18]:
comic_reviews_df.show(1)

+----------+--------------------+-------+-------------+--------------------+
|      asin|               imUrl|overall|   reviewerID|               title|
+----------+--------------------+-------+-------------+--------------------+
|0345507460|http://ecx.images...|    5.0|ACO26JQ366659|The Dresden Files...|
+----------+--------------------+-------+-------------+--------------------+
only showing top 1 row



In [19]:
comic_reviewers = comic_reviews_df.select('reviewerID').distinct().collect()

In [20]:
comic_reviewers = [r[0] for r in comic_reviewers]

In [21]:
video_reviews = video_reviews_df.filter(col('reviewerID').isin(comic_reviewers))

In [22]:
video_reviews.count()

5986

In [27]:
all_video_reviews = video_reviews.select(['asin', 'overall','reviewerID']).toPandas()

In [29]:
all_video_reviews['title'] = 'N/A - Amazon Instant Video'
all_video_reviews['imUrl'] = 'N/A - Amazon Instant Video'

In [36]:
all_video_reviews.shape

(5986, 5)

In [31]:
movie_reviews_df = spark.read.json('data/movie_reviews_wtitle.json').toPandas()

In [35]:
movie_reviews_df.shape

(103757, 5)

In [49]:
all_movies_df = pd.concat([movie_reviews_df, all_video_reviews], 
                           axis=0, sort=True).reset_index(drop=True)

In [51]:
all_movies_df.to_json('data/movie_and_video_wtitle.json')